In [1]:
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import dateutil.parser
from threading import Thread
%matplotlib qt

In [2]:
id = 1767
beginTime = '2017-05-20T18:22:53.253Z'
endTime = '2018-05-21T18:22:53.253Z'

In [3]:
class SNMPstream(object):
    
    def __init__(self, id, start, stop):
        self.id = id
        self.e1 = None
        self.e2 = None
        self.startdt = dateutil.parser.parse(start)
        self.stopdt = dateutil.parser.parse(stop)
        self.prefetched = self.fetch(self.startdt, self.stopdt)
        self.currblock = None
        self.current = None
        self.end = False
        self.prefetch()
        
    def __iter__(self): 
        return self
    
    def __next__(self):
        if self.startdt >= self.stopdt and self.end:
            raise StopIteration
        if self.prefetched == None:
            Thread(target=self.prefetch).start()
        if self.currblock == None:
            self.currblock = self.prefetched
        self.current = self.currblock.pop()
        print(self.current)
        return self.current
    
    def run_query(self, query): # A simple function to use requests to make the API call.
        request = requests.get('https://my.es.net/graphql', json={'query': query})
        if request.status_code == 200:
           return request.json()
        else:
           raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))
    
    def fetch(self, startdt, stopdt):
        beginTime = startdt.isoformat()[:-9] + 'Z'
        endTime = stopdt.isoformat()[:-9] + 'Z'
        query = '''
        {
          mapTopologyEdge(id: "%d") {
                name
            traffic(beginTime: "%s", endTime: "%s")
          }
        }
        ''' % (self.id, beginTime, endTime)
        print("fetching", beginTime, endTime)
        data = self.run_query(query)
        print(data)
        datastr = data["data"]["mapTopologyEdge"]["traffic"]
        self.e1, self.e2 = json.loads(datastr)["labels"]
        cleaned_data = np.array(json.loads(datastr)["points"])
        return cleaned_data
    
    def prefetch(self):
        if self.startdt + datetime.timedelta(1) < self.stopdt:
            self.prefetched = self.fetch(self.startdt, self.startdt + datetime.timedelta(1))
            self.startdt = self.startdt + datetime.timedelta(1)
        else:
            self.prefetched = self.fetch(self.startdt, self.stopdt)
            self.end = True

In [ ]:
def plot_stream(id, beginTime, endTime):    
    import matplotlib.pyplot as plt
    import time

    xdata = []
    ydata1 = []
    ydata2 = []
    datastream = SNMPstream(id, beginTime, endTime)
    
    plt.show()

    axes = plt.gca()
    line1, = axes.plot(xdata, ydata1, 'skyblue', linewidth=1)
    line2, = axes.plot(xdata, ydata2, 'olive', linewidth=1)
    line1.set_label(datastream.e1)
    line2.set_label(datastream.e2)
    axes.legend(loc=1)

    for i in datastream:
        if len(xdata) == 100:
            del xdata[0]
            del ydata1[0]
            del ydata2[0]
        xdata.append(i[0])
        ydata1.append(i[1])
        ydata2.append(i[2])
        line1.set_xdata(xdata)
        line1.set_ydata(ydata1)
        line2.set_xdata(xdata)
        line2.set_ydata(ydata2)
        axes.relim()
        axes.autoscale_view()
        plt.draw()
        plt.pause(1e-17)
        time.sleep(0.1)

    # add this if you don't want the window to disappear at the end
    plt.show()
plot_stream(id, beginTime, endTime)

fetching 2017-05-20T18:22:53.253Z 2018-05-21T18:22:53.253Z
